In [ ]:
import numpy as np
from cvxopt import solvers, matrix

We set the problem as follow : let $x$ be a vector 

We want to solve the following program :


$$\underset{c_1, c_2}{min}\hspace{0.3cm} \|c_1 \|_1 + \|c_2\|_1$$
$$s.t. x = \left[\Phi_1 \middle|  \Phi_2 \right] \begin{bmatrix}c_1 \\ c_2 \end{bmatrix} $$

The constraint can be rewritten 

We can rewrite it as a linear program

$$min \ \mathbb{1}^T ( c_1^{+} + c_2^{+} + c_1^{-} + c_2^{-}) $$

$$ s.t \hspace{1cm} 
\begin{aligned}
\begin{bmatrix} \Phi_1 & \Phi_2 & -\Phi_1 & -\Phi_2 \end{bmatrix} \begin{bmatrix} c_1\\ c_2 \\ c_1^{-} \\ c_2^{-} \end{bmatrix} = x\\
0 \leq c_i^{+}, 0 \leq c_i^{-}, i=1, 2
\end{aligned}$$

In [10]:
class DataSep():
    
    def __init__(self, basis1, basis2, x):
        self.basis1 = matrix(basis1) # ndarray of size n * d1
        self.basis2 = matrix(basis2) # ndarray of size n * d2
        self.x = matrix(x) # 1d array of size n 
        
    def coherence(self):
        """
        compute the mutual coherence between two systems
        """
        return 0
    def get_sparse_component(self):
        """
        solve the basis pursuit program
        """
        A = matrix(np.hstack((self.basis1, self.basis2, -self.basis1, -self.basis2)).T)
        b = self.x
        dim = A.shape[0]
        
        c = matrix( [1] * dim )
        G = -matrix(np.eye(dim))
        h = matrix([0] * dim)
        sol = cvxopt.solvers.lp(c, G, h, A, b)
        
        return sol
    
    

In [6]:
# to complete
basis1 = None
basis2 = None
data_sep = DataSep(basis1, basis2, x)